# Word Embeddings
- Using embedding layer in Keras
- Another way of learning word embeddings is via pre-training word vectors in another network (e.g., word2vec, GloVe, fasttext, etc.)
    - If you are interested in pretrained word embeddings, refer to: https://github.com/buomsoo-kim/Word-embedding-with-Python
    
<br>
<img src="https://adriancolyer.files.wordpress.com/2016/04/word2vec-distributed-representation.png?w=600" style="width: 600px"/>

## Word vectors
- Long story short, word embedding is process of converting each word to a fixed dimensional "(word) vector"
- Dimensionality of embedding space (i.e., vector space) is a hyperparameter; one can set dimensionality as any positive integer

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

### Embedding layer
- As a result, embedding layer bears 3-D tensors
    - Output shape = **(batch_size, input_length, output_dim)**
    - input_dim: dimensionality of input space (number of unique tokens of interest)
    - output_dim: dimensionality of embedding space
    - input_length: length of input sequence (if None, can vary

In [2]:
# when input length is constant
model = Sequential()
model.add(Embedding(input_dim = 10, output_dim = 5, input_length = 3))

In [3]:
model.output_shape

(None, 3, 5)

In [4]:
# when input length varies
model = Sequential()
model.add(Embedding(input_dim = 10, output_dim = 5, input_length = None))    

In [5]:
model.output_shape

(None, None, 5)

### Using embedding layer in network
- Usually, embedding layer are used as first layer in network to model text format data

In [6]:
# parameters to import dataset
num_words = 3000
maxlen = 50

In [7]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen)

2113536/2110848 [==============================] - 0s 0us/step


In [8]:
X_train = sequence.pad_sequences(X_train, maxlen = maxlen, padding = 'post')
X_test = sequence.pad_sequences(X_test, maxlen = maxlen, padding = 'post')

In [9]:
y_train = to_categorical(y_train, num_classes = 46)
y_test = to_categorical(y_test, num_classes = 46)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1595, 50)
(399, 50)
(1595, 46)
(399, 46)


In [11]:
input_dim = num_words
output_dim = 100     # we set dimensionality of embedding space as 100
input_length = maxlen

In [16]:
def reuters_model():
    model = Sequential()
    model.add(Embedding(input_dim = input_dim, output_dim = output_dim, input_length = input_length))
    model.add(GRU(50, return_sequences = False))
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dense(46, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [17]:
model = reuters_model()

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           300000    
_________________________________________________________________
gru (GRU)                    (None, 50)                22800     
_________________________________________________________________
dense (Dense)                (None, 100)               5100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 46)                4646      
Total params: 332,546
Trainable params: 332,546
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(X_train, y_train, epochs = 10, batch_size = 256, verbose = 1)

Epoch 1/10
7/7 [==============================] - 1s 82ms/step - loss: 1.1714 - accuracy: 0.7191
Epoch 2/10
7/7 [==============================] - 1s 83ms/step - loss: 1.1589 - accuracy: 0.7191
Epoch 3/10
7/7 [==============================] - 1s 83ms/step - loss: 1.1417 - accuracy: 0.7191
Epoch 4/10
7/7 [==============================] - 1s 81ms/step - loss: 1.1304 - accuracy: 0.7191
Epoch 5/10
7/7 [==============================] - 1s 82ms/step - loss: 1.1165 - accuracy: 0.7191
Epoch 6/10
7/7 [==============================] - 1s 87ms/step - loss: 1.1000 - accuracy: 0.7191
Epoch 7/10
7/7 [==============================] - 1s 86ms/step - loss: 1.0804 - accuracy: 0.7191
Epoch 8/10
7/7 [==============================] - 1s 84ms/step - loss: 1.0558 - accuracy: 0.7191
Epoch 9/10
7/7 [==============================] - 1s 85ms/step - loss: 1.0282 - accuracy: 0.7191
Epoch 10/10
7/7 [==============================] - 1s 82ms/step - loss: 1.0027 - accuracy: 0.7191


In [21]:
result = model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 7ms/step - loss: 0.9852 - accuracy: 0.7469


In [22]:
print('Test Accuracy: ', result[1])

Test Accuracy:  0.7468671798706055
